# Retriever tunner

A simple tool to compare and tune retriever performance, given a desired ranking to strive for. 
The goal is to provide a simple metric to measure how a given retriver is close to the 'ideal', generated for example
with a use of more expensive, slower or simply no-existant method. 

In [1]:
import sys
sys.path.append('../')
from python_modules.retriever_tunner import RetrieverTunner

### 1. Downloading test data

In [2]:
import nltk
from nltk.corpus import brown

# Download the necessary datasets from NLTK
nltk.download('brown')
nltk.download('punkt')

# Load the Brown Corpus as plain text
brown_corpus_text = ' '.join(brown.words())

# Split the corpus into sentences
sentences = nltk.sent_tokenize(brown_corpus_text)

# Display the number of sentences and first few sentences as a sample
print(f"Number of sentences: {len(sentences)}")
print("First few sentences:")
for sentence in sentences[:5]:
    print(sentence)

[nltk_data] Downloading package brown to
[nltk_data]     /Users/insani_dei/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/insani_dei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Number of sentences: 56601
First few sentences:
The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .
The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted .
The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. .
`` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' .
The jury said it did find that many of Georgia's registration and election laws `` are outmoded or inadequate and often ambiguous '' .


### 2. Initialize RetrieverTunner

In [3]:
rt = RetrieverTunner(
    # optional/ required
    search_values_list = sentences[0:1000],
    target_ranking_name = 'all-mpnet-base-v2',
    embedding_model_names = ['paraphrase-multilingual-mpnet-base-v2','all-mpnet-base-v2', 'multi-qa-mpnet-base-dot-v1', 'all-MiniLM-L6-v2'],
    # optionla
    similarity_search_h_params = {'processing_type' : 'parallel',
                                                  'max_workers' : 8,
                                                   'tbatch_size' : 1000},
    n_random_queries = 100,
    seed = 23,
    metrics_params = {'n_results' : [3,5,10],   
                      'prep_types' : ['correction', 'ceiling'],
                      'ratio' : 0.6,
                      'target_sum' : 1},
    # for plotting
    plots_params = {'top_n' : 3,
                    'text_lim' : 10,
                    'save_comp_plot' : False}
    )


### 3. Construct ranking and score

In [4]:
rt.construct_rankings()
rt.make_scores_dict()

{'all-mpnet-base-v2|paraphrase-multilingual-mpnet-base-v2': {'mean3correction': 77.05339871604721,
  'mean3ceiling': 1.1937121240807114,
  'mean5correction': 77.05339871604721,
  'mean5ceiling': 1.8648521209294744,
  'mean10correction': 77.05339871604721,
  'mean10ceiling': 3.3938589759760873,
  'mean100correction': 77.05339871604721,
  'mean100ceiling': 23.471943300857607},
 'all-mpnet-base-v2|multi-qa-mpnet-base-dot-v1': {'mean3correction': 61.92952825621033,
  'mean3ceiling': 1.20764917582372,
  'mean5correction': 61.92952825621033,
  'mean5ceiling': 1.901777077449774,
  'mean10correction': 61.92952825621033,
  'mean10ceiling': 3.4283081004367157,
  'mean100correction': 61.92952825621033,
  'mean100ceiling': 21.042288154158854},
 'all-mpnet-base-v2|all-MiniLM-L6-v2': {'mean3correction': 75.43242957785465,
  'mean3ceiling': 1.2313563782963435,
  'mean5correction': 75.43242957785465,
  'mean5ceiling': 1.9351671773764914,
  'mean10correction': 75.43242957785465,
  'mean10ceiling': 3.49

In [5]:
rt.ranking_dicts['all-MiniLM-L6-v2']

{"The detailed rundown on the Kansas City scoring in the sixth went like this : Lumpe worked a walk as the first batter to face Hyde and romped around as Siebern blasted Hyde's next toss 415 feet over the scoreboard in right center .": [947,
  929,
  944,
  940,
  945,
  925,
  924,
  923,
  928,
  936,
  946,
  926,
  993,
  941,
  938,
  932,
  948,
  933,
  965,
  880,
  973,
  954,
  996,
  991,
  930,
  972,
  937,
  949,
  927,
  955,
  942,
  994,
  935,
  939,
  934,
  797,
  990,
  983,
  966,
  931,
  950,
  62,
  921,
  858,
  943,
  798,
  922,
  476,
  806,
  962,
  148,
  975,
  963,
  848,
  964,
  984,
  967,
  995,
  978,
  469,
  175,
  704,
  808,
  961,
  958,
  781,
  2,
  985,
  437,
  804,
  986,
  202,
  153,
  107,
  461,
  802,
  807,
  687,
  600,
  216,
  974,
  571,
  49,
  960,
  166,
  663,
  192,
  94,
  915,
  553,
  364,
  544,
  863,
  272,
  229,
  969,
  85,
  106,
  529,
  218,
  871,
  782,
  998,
  653,
  12,
  144,
  968,
  212,
  206,
  979,
  

### 4. Plot rankings

In [9]:
rt.show_model_comparison_plots(top_n=2)